In [1]:
## imporrt libarary 
import os
import pandas as pd
import numpy as np

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform
from support_functions import missing_value, fill_missing, list_dtypes

## sklearn module
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading Model
filename = 'models/California-House-Price-XGB-ML-Regression-Model.sav'
loaded_model = pickle.load(open(filename,'rb'))

# Sample data for prediction (row 5 & 99)
new_data = pd.DataFrame({
    'longitude': [-117.91, -122.38],
    'latitude': [34.12, 39.68], 
    'housing_median_age': [41, 21],
    'total_rooms': [2673, 1155], 
    'total_bedrooms': [578, 210], 
    'population': [2259, 510], 
    'households': [592, 175], 
    'median_income': [3.7846, 2.3851], 
    'ocean_proximity': ["<1H OCEAN", "INLAND"], 
    'income_cat': ["Upper-Middle Class", "Middle Class"], 
    'housingAge_cat': ["Prehistoric", "Middle-Aged"]
})

# Predict using the loaded model
new_predictions = loaded_model.predict(new_data)
print("Predictions for new data:", new_predictions)


Predictions for new data: [190281.86  90383.77]


In [3]:
project_id = 'dti-ds'
dataset_id = 'rizqi_dataset_017'
table_id = 'california_house_test' # where it loads the "california_house_test.csv" from local -> local file containing X-test only (2200 rows etc)
region = 'us-central1'
bucket_name = 'rizqi_gcs_017'

In [4]:
# load data from BQ
client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
test_df = query_job.result().to_dataframe()

test_df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,income_cat,housingAge_cat
0,-118.28,33.94,9,456,130,438,114,0.8952,<1H OCEAN,Poor,New
1,-120.17,39.33,10,614,141,195,95,0.9283,INLAND,Poor,New
2,-120.30,38.90,11,1961,435,113,53,0.9227,INLAND,Poor,New
3,-117.10,33.12,12,961,342,315,201,0.8130,<1H OCEAN,Poor,New
4,-118.24,34.05,13,1703,697,1823,669,0.8288,<1H OCEAN,Poor,New
...,...,...,...,...,...,...,...,...,...,...,...
2608,-118.32,34.08,52,1137,304,754,297,3.3700,<1H OCEAN,Upper-Middle Class,Prehistoric
2609,-118.25,34.09,52,3142,765,1728,682,3.1864,<1H OCEAN,Upper-Middle Class,Prehistoric
2610,-118.12,34.17,52,1835,330,777,317,3.7159,INLAND,Upper-Middle Class,Prehistoric
2611,-122.41,37.80,52,1724,416,1016,395,3.3839,NEAR BAY,Upper-Middle Class,Prehistoric


In [6]:
y_pred = loaded_model.predict(test_df)
y_pred_df = pd.DataFrame(y_pred, columns=['median_house_value'])

result_df = pd.concat([test_df.reset_index(drop=True), y_pred_df.reset_index(drop=True)], axis=1)
result_df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,income_cat,housingAge_cat,median_house_value
0,-118.28,33.94,9,456,130,438,114,0.8952,<1H OCEAN,Poor,New,140675.109375
1,-120.17,39.33,10,614,141,195,95,0.9283,INLAND,Poor,New,121074.820312
2,-120.30,38.90,11,1961,435,113,53,0.9227,INLAND,Poor,New,129979.218750
3,-117.10,33.12,12,961,342,315,201,0.8130,<1H OCEAN,Poor,New,143323.203125
4,-118.24,34.05,13,1703,697,1823,669,0.8288,<1H OCEAN,Poor,New,220072.531250
...,...,...,...,...,...,...,...,...,...,...,...,...
2608,-118.32,34.08,52,1137,304,754,297,3.3700,<1H OCEAN,Upper-Middle Class,Prehistoric,260934.453125
2609,-118.25,34.09,52,3142,765,1728,682,3.1864,<1H OCEAN,Upper-Middle Class,Prehistoric,232845.281250
2610,-118.12,34.17,52,1835,330,777,317,3.7159,INLAND,Upper-Middle Class,Prehistoric,239814.421875
2611,-122.41,37.80,52,1724,416,1016,395,3.3839,NEAR BAY,Upper-Middle Class,Prehistoric,309914.625000


In [7]:
from google.cloud.exceptions import NotFound

dest_table_id = 'california_house_prediction'
table_full_id = f'{project_id}.{dataset_id}.{dest_table_id}'

schema = [
    bigquery.SchemaField('longitude', 'FLOAT'),
    bigquery.SchemaField('latitude', 'FLOAT'),
    bigquery.SchemaField('housing_median_age', 'INTEGER'),
    bigquery.SchemaField('total_rooms', 'INTEGER'),
    bigquery.SchemaField('total_bedrooms', 'INTEGER'),
    bigquery.SchemaField('population', 'INTEGER'),
    bigquery.SchemaField('households', 'INTEGER'),
    bigquery.SchemaField('median_income', 'FLOAT'),
    bigquery.SchemaField('ocean_proximity', 'STRING'),
    bigquery.SchemaField('income_cat', 'STRING'),
    bigquery.SchemaField('housingAge_cat', 'STRING'),
    bigquery.SchemaField('median_house_value', 'FLOAT')
]

table_ref = client.dataset(dataset_id).table(dest_table_id)

# Check if the table exists
try:
    client.get_table(table_ref)
    print(f'Table {table_full_id} already exists.')
except NotFound:
    # Create the table if it does not exist
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)
    print(f'Table {table_full_id} created.')

job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)

load_job = client.load_table_from_dataframe(result_df, table_ref, job_config=job_config)
load_job.result()

print(f'Loaded {load_job.output_rows} rows into {table_full_id}')

Table dti-ds.rizqi_dataset_017.california_house_prediction already exists.
Loaded 2613 rows into dti-ds.rizqi_dataset_017.california_house_prediction
